In [3]:
# necessary imports
import os
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv

# load huggingface api key
load_dotenv()

True

In [39]:
# sample document
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [40]:
# embedding model
embedding_model = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

# LLM
llm = ChatGroq(model="llama-3.1-8b-instant")

In [41]:
# vector store
vector_store = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embedding_model,
    index_name=os.getenv("PINECONE_INDEX_NAME")
)

In [42]:
# base retriever
base_retriever = vector_store.as_retriever(search_kwargs={"k":2})

# base compressor
base_compressor = LLMChainExtractor.from_llm(llm=llm)

In [43]:
# user query 
query = "what isPhotosynthesis"

In [44]:
# contextual compression retriever
contextual_compression_retriever = ContextualCompressionRetriever(
    base_compressor=base_compressor,
    base_retriever=base_retriever
)

In [47]:
# final result - retrieved document
result = contextual_compression_retriever.invoke(input=query)
for i, res in enumerate(result, start=1):
    print(f"\n----- Document:{i} -----\n")
    print(res.page_content)


----- Document:1 -----

Photosynthesis enables plants to produce energy by converting sunlight.

----- Document:2 -----

Photosynthesis does not occur in animal cells.
